In [371]:
import numpy as np
import pandas as pd
import itertools

In [372]:
def get_dayclose(dataset, name):
    data_dayclose = dataset.loc[:,["Date","Close"]]
    dayclose_name = name + " close price"
    data_dayclose.columns = ["Date", dayclose_name]
    return data_dayclose

def get_diff(dataset, name):
    dataset["diff"] = dataset["Close"] - dataset["Open"]
    data_diff = dataset.loc[:,["Date","diff"]]
    diff_name = name + " diff"
    data_diff.columns = ["Date", diff_name]
    return data_diff


data_APPL = pd.read_csv("AAPL.csv")

data_APPL_dayclose = get_dayclose(data_APPL, "APPL")
data_APPL_diff = get_diff(data_APPL, "APPL")
print(data_APPL_dayclose.shape)

data_MSFT = pd.read_csv("MSFT.csv")
print(data_MSFT.shape)
data_MSFT_dayclose = get_dayclose(data_MSFT, "MSFT")
data_MSFT_diff = get_diff(data_MSFT, "MSFT")


data_QCOM = pd.read_csv("QCOM_T.csv")

data_QCOM_dayclose = get_dayclose(data_QCOM, "QCOM")
data_QCOM_diff = get_diff(data_MSFT, "QCOM")


data_TXN = pd.read_csv("TXN.csv")

data_TXN_dayclose = get_dayclose(data_TXN, "TXN")
data_TXN_diff = get_diff(data_MSFT, "TXN")

data_all_closeprice = pd.merge(data_TXN_dayclose,data_QCOM_dayclose)
data_all_closeprice = pd.merge(data_all_closeprice,data_MSFT_dayclose)
data_all_closeprice = pd.merge(data_all_closeprice,data_APPL_dayclose)
data_all_closeprice.to_csv("all close price.csv")


data_all_diff = pd.merge(data_TXN_diff,data_QCOM_diff)

data_all_diff = pd.merge(data_all_diff,data_MSFT_diff)
data_all_diff = pd.merge(data_all_diff,data_APPL_diff)


(1007, 2)
(1007, 7)


In [373]:
data_exchange = pd.read_csv("daily.csv")

def exchange_rate_contry(dataset, country):
    data_con = dataset.loc[data_exchange["Country"] == country]
    data_con = data_con.drop(columns = ["Country"])
    data_con = data_con.set_index("Date")
    data_con = data_con["2014-01-01":"2017-12-31"]
    date = data_con.index
    ind = np.arange(data_con.shape[0])
    data_con = data_con.set_index(ind)
    data_con["Date"] = date
    exchange_name = country + " exchange rate"
    data_con.columns = [exchange_name, "Date"]
    return data_con

data_exchange_chn = exchange_rate_contry(data_exchange, "China")

data_exchange_can = exchange_rate_contry(data_exchange, "Canada")

data_exchange_euro = exchange_rate_contry(data_exchange, "Euro")    

data_exchange_hk = exchange_rate_contry(data_exchange, "Hong Kong")

# data_exchange_hk.head(5)


data_all_exchangerate = pd.merge(data_exchange_euro, data_exchange_can)
data_all_exchangerate = pd.merge(data_all_exchangerate, data_exchange_chn)
data_all_exchangerate = pd.merge(data_all_exchangerate, data_exchange_hk)

#switch order
cols = data_all_exchangerate.columns.tolist()
a = cols[0]
cols[0] = cols[1]
cols[1] = a
data_all_exchangerate = data_all_exchangerate[cols]
data_all_exchangerate.head(5)

,Date,Euro exchange rate,Canada exchange rate,China exchange rate,Hong Kong exchange rate
0,2014-01-01,NaN,NaN,NaN,NaN
1,2014-01-02,0.731529,1.0634,6.0504,7.7534
2,2014-01-03,0.734970,1.0612,6.0505,7.7537
3,2014-01-06,0.733353,1.0658,6.0524,7.7542
4,2014-01-07,0.734322,1.0742,6.0507,7.7542


In [374]:
data_stock_exchange = pd.merge(data_all_exchangerate, data_all_closeprice)
data_diff_exchange = pd.merge(data_all_exchangerate, data_all_diff)
print(data_diff_exchange.shape)

def take_away_zeros(date):
    if len(date) == 10:
        if date[5] == "0":
            if date[8] == "0":
                date = date[0:5] + date[6:8] + date[9:]
            else:
                date = date[0:5] + date[6:]
        elif date[8] == "0":
            date = date[0:8] + date[9:]
    return date
# data_stock_exchange["Date"] = data_stock_exchange["Date"].apply(take_away_zeros)
# data_stock_exchange.head(5)
data_stock_exchange.head()

(1007, 9)


,Date,Euro exchange rate,Canada exchange rate,China exchange rate,Hong Kong exchange rate,TXN close price,QCOM close price,MSFT close price,APPL close price
0,2014-01-02,0.731529,1.0634,6.0504,7.7534,43.099998,73.320000,37.160000,79.018570
1,2014-01-03,0.734970,1.0612,6.0505,7.7537,43.290001,72.889999,36.910000,77.282860
2,2014-01-06,0.733353,1.0658,6.0524,7.7542,42.930000,72.699997,36.130001,77.704285
3,2014-01-07,0.734322,1.0742,6.0507,7.7542,42.700001,73.239998,36.410000,77.148575
4,2014-01-08,0.736052,1.0802,6.0510,7.7543,43.290001,73.680000,35.759998,77.637146


In [375]:

data_stock_exchange["Date"] = data_stock_exchange["Date"].apply(take_away_zeros)
data_all_closeprice["Date"] = data_all_closeprice["Date"].apply(take_away_zeros)
data_diff_exchange["Date"] = data_diff_exchange["Date"].apply(take_away_zeros)
print(data_stock_exchange.shape)



(1007, 9)


In [376]:

data_news = pd.read_csv("news_in_days_with_three_parameters.csv")
data_all_news = pd.read_csv("news_in_days_with_all_parameters.csv")

data_news = data_news.drop(columns = ["Unnamed: 0"])
data_all_news = data_all_news.drop(columns = ["Unnamed: 0"])

data_news = data_news.rename(columns={"date": "Date"})
data_all_news = data_all_news.rename(columns={"date-month-day": "Date"})

print(data_news.shape)
print(data_all_news.shape)
data_all_news.head(5)




(1105, 5)
(1131, 32)


,social_media,computer,business,programming,hearing,urban,shopping,science,work,valuable,...,horror,suffering,ridicule,irritability,deception,disappointment,negative_emotion,nervousness,appear,Date
0,0.001504,0.000000,0.003008,0.000000,0.013534,0.000000,0.000000,0.001504,0.001504,0.004511,...,0.0,0.001504,0.004511,0.0,0.001504,0.0,0.001504,0.001504,1.0,2000-05-15
1,0.001436,0.002154,0.002154,0.002154,0.000718,0.002872,0.005025,0.000000,0.001436,0.002154,...,0.0,0.002872,0.000000,0.0,0.000000,0.0,0.007897,0.002154,1.0,2003-06-14
2,0.000978,0.002935,0.002935,0.000000,0.001957,0.000978,0.000978,0.000978,0.003914,0.000000,...,0.0,0.000000,0.000000,0.0,0.001957,0.0,0.003914,0.001957,1.0,2003-09-19
3,0.000000,0.000000,0.010267,0.000000,0.000000,0.002053,0.004107,0.010267,0.006160,0.006160,...,0.0,0.008214,0.000000,0.0,0.004107,0.0,0.008214,0.000000,1.0,2004-06-06
4,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.111111,0.000000,...,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,1.0,2004-11-18


In [377]:
def convert_time(old_date):
    x = []
    for i in range(len(old_date)):
        from datetime import datetime, date
        c = str(old_date[i])
        y,m,d = c.split("-")
        try:
            utc_date =date(int(y), int(m), int(d))
            DAY = 24*60*60
            timestamp = (utc_date.toordinal() - date(1970, 1, 1).toordinal()) * DAY

            timestamp = (utc_date- date(1970, 1, 1)).days * DAY
            date = datetime.utcfromtimestamp(timestamp).strftime('%Y-%m-%d')
            x.append(date)
        except:
            print(c)
            print("strange error!")
            x.append(c)
    return x

In [378]:
a = data_news["Date"].tolist()

x = convert_time(a)

data_news["Date"] = x
data_news = data_news.sort_values(by='Date')
data_news.head(100)
ind = np.arange(data_news.shape[0])
data_news = data_news.set_index(ind)

ind = np.arange(data_all_news.shape[0])
data_all_news = data_all_news.set_index(ind)
data_all_news.head(5)


date-month-day
我操你妈逼垃圾狗屎error


,social_media,computer,business,programming,hearing,urban,shopping,science,work,valuable,...,horror,suffering,ridicule,irritability,deception,disappointment,negative_emotion,nervousness,appear,Date
0,0.001504,0.000000,0.003008,0.000000,0.013534,0.000000,0.000000,0.001504,0.001504,0.004511,...,0.0,0.001504,0.004511,0.0,0.001504,0.0,0.001504,0.001504,1.0,2000-05-15
1,0.001436,0.002154,0.002154,0.002154,0.000718,0.002872,0.005025,0.000000,0.001436,0.002154,...,0.0,0.002872,0.000000,0.0,0.000000,0.0,0.007897,0.002154,1.0,2003-06-14
2,0.000978,0.002935,0.002935,0.000000,0.001957,0.000978,0.000978,0.000978,0.003914,0.000000,...,0.0,0.000000,0.000000,0.0,0.001957,0.0,0.003914,0.001957,1.0,2003-09-19
3,0.000000,0.000000,0.010267,0.000000,0.000000,0.002053,0.004107,0.010267,0.006160,0.006160,...,0.0,0.008214,0.000000,0.0,0.004107,0.0,0.008214,0.000000,1.0,2004-06-06
4,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.111111,0.000000,...,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,1.0,2004-11-18


In [379]:
close_news_diff = pd.merge(data_APPL_diff, data_news, on = "Date", how = "right")
close_news_diff = pd.merge(data_TXN_diff, close_news_diff, on = "Date", how = "right")
close_news_diff = pd.merge(data_MSFT_diff, close_news_diff, on = "Date", how = "right")
close_news_diff = pd.merge(data_QCOM_diff, close_news_diff, on = "Date", how = "right")
close_news_diff = pd.merge(data_exchange_chn, close_news_diff, on = "Date", how = "right")
close_news_diff = pd.merge(data_exchange_can, close_news_diff, on = "Date", how = "right")
close_news_diff = pd.merge(data_exchange_hk, close_news_diff, on = "Date", how = "right")
close_news_diff = pd.merge(data_exchange_euro, close_news_diff, on = "Date", how = "right")

data_all_news

data_all_news_diff = pd.merge(data_APPL_diff, data_all_news, on = "Date", how = "right")
data_all_news_diff = pd.merge(data_TXN_diff, data_all_news_diff, on = "Date", how = "right")
data_all_news_diff = pd.merge(data_MSFT_diff, data_all_news_diff, on = "Date", how = "right")
data_all_news_diff = pd.merge(data_QCOM_diff, data_all_news_diff, on = "Date", how = "right")
data_all_news_diff = pd.merge(data_exchange_chn, data_all_news_diff, on = "Date", how = "right")
data_all_news_diff = pd.merge(data_exchange_can, data_all_news_diff, on = "Date", how = "right")
data_all_news_diff = pd.merge(data_exchange_hk, data_all_news_diff, on = "Date", how = "right")
data_all_news_diff = pd.merge(data_exchange_euro, data_all_news_diff, on = "Date", how = "right")

print(close_news_diff.shape)
close_news_diff = close_news_diff.sort_values(by = "Date")
close_news_diff = close_news_diff.sort_values(by = "Date")


close_news_diff = close_news_diff.iloc[182:,:]
close_news_diff = close_news_diff.iloc[182:,:]

ind = np.arange(close_news_diff.shape[0])
close_news_diff = close_news_diff.set_index(ind)

ind = np.arange(close_news_diff.shape[0])
close_news_diff = close_news_diff.set_index(ind)

close_news_diff.to_csv("testmerge2.csv")
data_all_news.head(5)




(1105, 13)


,social_media,computer,business,programming,hearing,urban,shopping,science,work,valuable,...,horror,suffering,ridicule,irritability,deception,disappointment,negative_emotion,nervousness,appear,Date
0,0.001504,0.000000,0.003008,0.000000,0.013534,0.000000,0.000000,0.001504,0.001504,0.004511,...,0.0,0.001504,0.004511,0.0,0.001504,0.0,0.001504,0.001504,1.0,2000-05-15
1,0.001436,0.002154,0.002154,0.002154,0.000718,0.002872,0.005025,0.000000,0.001436,0.002154,...,0.0,0.002872,0.000000,0.0,0.000000,0.0,0.007897,0.002154,1.0,2003-06-14
2,0.000978,0.002935,0.002935,0.000000,0.001957,0.000978,0.000978,0.000978,0.003914,0.000000,...,0.0,0.000000,0.000000,0.0,0.001957,0.0,0.003914,0.001957,1.0,2003-09-19
3,0.000000,0.000000,0.010267,0.000000,0.000000,0.002053,0.004107,0.010267,0.006160,0.006160,...,0.0,0.008214,0.000000,0.0,0.004107,0.0,0.008214,0.000000,1.0,2004-06-06
4,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.111111,0.000000,...,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,1.0,2004-11-18


In [380]:
close_news = pd.merge(data_APPL_dayclose, data_news, on = "Date", how = "right")
close_news = pd.merge(data_TXN_dayclose, close_news, on = "Date", how = "right")
close_news = pd.merge(data_MSFT_dayclose, close_news, on = "Date", how = "right")
close_news = pd.merge(data_QCOM_dayclose, close_news, on = "Date", how = "right")
close_news = pd.merge(data_exchange_chn, close_news, on = "Date", how = "right")
close_news = pd.merge(data_exchange_can, close_news, on = "Date", how = "right")
close_news = pd.merge(data_exchange_hk, close_news, on = "Date", how = "right")
close_news = pd.merge(data_exchange_euro, close_news, on = "Date", how = "right")




data_all_news= pd.merge(data_APPL_dayclose, data_all_news, on = "Date", how = "right")
data_all_news = pd.merge(data_TXN_dayclose, data_all_news, on = "Date", how = "right")
data_all_news = pd.merge(data_MSFT_dayclose, data_all_news, on = "Date", how = "right")
data_all_news = pd.merge(data_QCOM_dayclose, data_all_news, on = "Date", how = "right")
data_all_news = pd.merge(data_exchange_chn, data_all_news, on = "Date", how = "right")
data_all_news = pd.merge(data_exchange_can, data_all_news, on = "Date", how = "right")
data_all_news = pd.merge(data_exchange_hk, data_all_news, on = "Date", how = "right")
data_all_news = pd.merge(data_exchange_euro, data_all_news, on = "Date", how = "right")


print(data_all_news.shape)
close_news = close_news.sort_values(by = "Date")
data_all_news = data_all_news.sort_values(by = "Date")


close_news = close_news.iloc[182:,:]
data_all_news = data_all_news.iloc[199:,:]

ind = np.arange(close_news.shape[0])
close_news = close_news.set_index(ind)

ind = np.arange(data_all_news.shape[0])
data_all_news = data_all_news.set_index(ind)

close_news.to_csv("testmerge1.csv")
data_all_news.head(5)

(1131, 40)


,Euro exchange rate,Date,Hong Kong exchange rate,Canada exchange rate,China exchange rate,QCOM close price,MSFT close price,TXN close price,APPL close price,social_media,...,aggression,horror,suffering,ridicule,irritability,deception,disappointment,negative_emotion,nervousness,appear
0,0.734322,2014-01-07,7.7542,1.0742,6.0507,73.239998,36.410000,42.700001,77.148575,0.000000,...,0.001553,0.000000,0.000000,0.000000,0.0,0.001553,0.001553,0.000000,0.000000,1.0
1,0.730887,2014-01-14,7.7536,1.0934,6.0402,73.389999,35.779999,43.340000,78.055717,0.000000,...,0.001570,0.000000,0.000000,0.000000,0.0,0.001570,0.000000,0.009419,0.000000,1.0
2,0.735078,2014-01-15,7.7543,1.0932,6.0459,74.510002,36.759998,43.540001,79.622856,0.001441,...,0.005764,0.000000,0.000000,0.001441,0.0,0.001441,0.002882,0.000000,0.007205,1.0
3,NaN,2014-01-19,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000886,...,0.003101,0.000443,0.003101,0.000443,0.0,0.000886,0.001329,0.005760,0.000443,1.0
4,NaN,2014-01-20,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,...,0.003387,0.000847,0.005080,0.000000,0.0,0.000847,0.000000,0.005080,0.004234,1.0


In [381]:


close_news["new_correlation"] = np.nan
close_news["pos_re"] = np.nan
close_news["neg_re"] = np.nan
close_news["appearence"] = np.nan
print(close_news.shape)
ind = np.arange(close_news.shape[0])
close_news = close_news.set_index(ind)
close_news.to_csv("beforeconv.csv")
print(close_news_diff.shape)

(923, 17)
(923, 13)


In [382]:
def shift_news(close_news_test, name = "", column_name = ""):
    #create new columns
    close_news_test["new_correlation"] = np.nan
    close_news_test["pos_re"] = np.nan
    close_news_test["neg_re"] = np.nan
    close_news_test["appearence"] = np.nan
    #get the first value
    appearence = close_news_test["apperance"][0]
    corr = close_news_test["correlation"][0]
    pos_re = close_news_test["positive_relavent"][0]
    neg_re = close_news_test["negetive_relavent"][0]
    for i in range(1, close_news_test.shape[0]):
        #possible index error exceeding the maximum
        #today has stock price and tomorrow does not
        #put the value into variable
        if np.isnan(close_news_test[column_name][i]) == False:
            close_news_test["new_correlation"][i] = corr / appearence
            close_news_test["pos_re"][i] = pos_re / appearence
            close_news_test["neg_re"][i] = neg_re / appearence
            close_news_test["appearence"][i] = appearence
            corr = close_news_test["correlation"][i]
            pos_re = close_news_test["positive_relavent"][i]
            neg_re = close_news_test["negetive_relavent"][i]
            appearence = close_news_test["apperance"][i]

        elif np.isnan(close_news_test[column_name][i]) == True:
            corr += close_news_test["correlation"][i]
            pos_re += close_news_test["positive_relavent"][i]
            neg_re += close_news_test["negetive_relavent"][i]
            appearence += close_news_test["apperance"][i]
    whole_name = name + ".csv"
    close_news_test.to_csv(whole_name)

In [383]:
shifted = shift_news(close_news, "close_news", "APPL close price")
shifted2 = shift_news(close_news_diff, "diff", "APPL diff")

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Library/Frameworks/Python.framework/Versions/

In [388]:
ind = np.arange(data_all_news.shape[0])
data_all_news = data_all_news.set_index(ind)

lst_column = list(data_all_news.columns)

di_column = lst_column[2:]
x = data_all_news.iloc[0:1,:]
x.head()
lst_first_val = []
for key in di_column:
    lst_first_val.append(x[key][0])

adict = dict(zip(di_column,lst_first_val))
print(adict)
data_all_news.head(100)

{'Hong Kong exchange rate': 7.7542, 'Canada exchange rate': 1.0742, 'China exchange rate': 6.0507, 'QCOM close price': 73.239998, 'MSFT close price': 36.41, 'TXN close price': 42.700001, 'APPL close price': 77.148575, 'social_media': 0.0, 'computer': 0.0, 'business': 0.003105590062, 'programming': 0.0, 'hearing': 0.001552795031, 'urban': 0.001552795031, 'shopping': 0.0, 'science': 0.010869565217, 'work': 0.0, 'valuable': 0.0, 'fashion': 0.0, 'technology': 0.0, 'competing': 0.001552795031, 'economics': 0.0, 'office': 0.0, 'cheerfulness': 0.0, 'optimism': 0.0, 'celebration': 0.003105590062, 'trust': 0.0, 'positive_emotion': 0.0, 'hate': 0.0, 'aggression': 0.001552795031, 'horror': 0.0, 'suffering': 0.0, 'ridicule': 0.0, 'irritability': 0.0, 'deception': 0.001552795031, 'disappointment': 0.001552795031, 'negative_emotion': 0.0, 'nervousness': 0.0, 'appear': 1.0}


,Euro exchange rate,Date,Hong Kong exchange rate,Canada exchange rate,China exchange rate,QCOM close price,MSFT close price,TXN close price,APPL close price,social_media,...,aggression,horror,suffering,ridicule,irritability,deception,disappointment,negative_emotion,nervousness,appear
0,0.734322,2014-01-07,7.7542,1.0742,6.0507,73.239998,36.41,42.700001,77.148575,0.000000,...,0.001553,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,1.552795e-03,1.552795e-03,0.000000,0.000000,1.0
1,0.730887,2014-01-14,7.7542,1.0742,6.0507,73.239998,36.41,42.700001,77.148575,0.000000,...,0.001553,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,1.552795e-03,1.552795e-03,0.000000,0.000000,1.0
2,0.735078,2014-01-15,7.7542,1.0742,6.0507,73.239998,36.41,42.700001,77.148575,0.000000,...,0.001553,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,1.552795e-03,1.552795e-03,0.000000,0.000000,1.0
3,NaN,2014-01-19,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000886,...,0.003101,4.430660e-04,0.003101,4.430660e-04,0.000000e+00,8.861320e-04,1.329198e-03,0.005760,0.000443,1.0
4,NaN,2014-01-20,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,...,0.003387,8.467401e-04,0.005080,0.000000e+00,0.000000e+00,8.467401e-04,0.000000e+00,0.005080,0.004234,1.0
5,0.734538,2014-02-07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000295,...,0.002680,4.299354e-04,0.002727,1.476887e-04,0.000000e+00,1.095222e-03,9.606644e-04,0.003613,0.001559,3.0
6,0.732923,2014-02-10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000295,...,0.002680,4.299354e-04,0.002727,1.476887e-04,0.000000e+00,1.095222e-03,9.606644e-04,0.003613,0.001559,1.0
7,NaN,2014-02-17,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.002326,...,0.002326,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.002326,0.000000,1.0
8,0.730460,2014-02-20,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.001310,...,0.002503,2.149677e-04,0.001364,7.384434e-05,0.000000e+00,5.476112e-04,4.803322e-04,0.002970,0.000779,2.0
9,0.727590,2014-02-24,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.001310,...,0.002503,2.149677e-04,0.001364,7.384434e-05,0.000000e+00,5.476112e-04,4.803322e-04,0.002970,0.000779,1.0


In [385]:
new_data_all_news1 = data_all_news.iloc[:, :]
new_data_all_news2 = new_data_all_news1
# new_data_all_news2["APPL close price"][8]
print(new_data_all_news2.shape)
new_data_all_news2.head()

(932, 40)


,Euro exchange rate,Date,Hong Kong exchange rate,Canada exchange rate,China exchange rate,QCOM close price,MSFT close price,TXN close price,APPL close price,social_media,...,aggression,horror,suffering,ridicule,irritability,deception,disappointment,negative_emotion,nervousness,appear
0,0.734322,2014-01-07,7.7542,1.0742,6.0507,73.239998,36.410000,42.700001,77.148575,0.000000,...,0.001553,0.000000,0.000000,0.000000,0.0,0.001553,0.001553,0.000000,0.000000,1.0
1,0.730887,2014-01-14,7.7536,1.0934,6.0402,73.389999,35.779999,43.340000,78.055717,0.000000,...,0.001570,0.000000,0.000000,0.000000,0.0,0.001570,0.000000,0.009419,0.000000,1.0
2,0.735078,2014-01-15,7.7543,1.0932,6.0459,74.510002,36.759998,43.540001,79.622856,0.001441,...,0.005764,0.000000,0.000000,0.001441,0.0,0.001441,0.002882,0.000000,0.007205,1.0
3,NaN,2014-01-19,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000886,...,0.003101,0.000443,0.003101,0.000443,0.0,0.000886,0.001329,0.005760,0.000443,1.0
4,NaN,2014-01-20,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,...,0.003387,0.000847,0.005080,0.000000,0.0,0.000847,0.000000,0.005080,0.004234,1.0


In [386]:
for i in range(1, new_data_all_news1.shape[0]):
    if np.isnan(new_data_all_news2["APPL close price"][i]) == False:
        for key, value in adict.items():
            if key != "appear":


                new_data_all_news2[key][i] = value / adict["appear"]
            

                
                adict[key] = new_data_all_news1[key][i]
            else:
                adict[key] = new_data_all_news1[key][i]
                new_data_all_news2[key][i] = value


    elif np.isnan(new_data_all_news2["APPL close price"][i]) == True:
        for key, value in adict.items():
            if key != "appear":
                adict[key] += new_data_all_news1[key][i]

            else:
                adict[key] += new_data_all_news1[key][i]



/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [387]:
new_data_all_news2.to_csv("close_all_news_extra.csv")

In [12]:

data_all_merge = pd.merge(data_stock_exchange, data_news)

data_diff_merge = pd.merge(data_diff_exchange, data_news)


print(data_all_merge.shape)

(654, 13)


In [13]:
print(data_all_merge.shape)
print(data_diff_merge.shape)
data_all_merge.to_csv("merged_data.csv", encoding='utf-8')
data_diff_merge.to_csv("data_diff_merge.csv",encoding='utf-8')
data_all_merge.head(44)



(654, 13)
(654, 13)


,Date,Euro exchange rate,Canada exchange rate,China exchange rate,Hong Kong exchange rate,TXN close price,QCOM close price,MSFT close price,APPL close price,correlation,positive_relavent,negetive_relavent,apperance
0,2014-1-3,0.734970,1.0612,6.0505,7.7537,43.290001,72.889999,36.910000,77.282860,0.477528,0.002012,0.000402,1.0
1,2014-1-7,0.734322,1.0742,6.0507,7.7542,42.700001,73.239998,36.410000,77.148575,0.124224,0.000579,0.000116,1.0
2,2014-1-14,0.730887,1.0934,6.0402,7.7536,43.340000,73.389999,35.779999,78.055717,0.115123,0.001446,0.000289,1.0
3,2014-1-15,0.735078,1.0932,6.0459,7.7543,43.540001,74.510002,36.759998,79.622856,0.211335,0.003959,0.000792,1.0
4,2014-2-7,0.734538,1.1014,6.0629,7.7586,41.950001,74.410004,36.560001,74.239998,0.171380,0.002056,0.000411,1.0
5,2014-2-10,0.732923,1.1048,6.0591,7.7569,42.360001,74.690002,36.799999,75.570000,0.203837,0.003666,0.000733,1.0
6,2014-2-24,0.727590,1.1058,6.0982,7.7591,44.450001,75.430000,37.689999,75.364288,0.119048,0.000709,0.000142,1.0
7,2014-3-3,0.726586,1.1090,6.1460,7.7600,44.580002,73.629997,37.779999,75.394287,0.077295,0.000000,0.000000,1.0
8,2014-3-12,0.719217,1.1130,6.1448,7.7657,46.430000,76.970001,38.270000,76.658569,0.289389,0.000931,0.000186,1.0
9,2014-3-13,0.718030,1.1059,6.1359,7.7644,44.990002,75.629997,37.889999,75.807144,0.114943,0.000743,0.000149,1.0


In [393]:
all_p = pd.read_csv("allnewsfinal.csv")

In [390]:
all_p = pd.merge(data_TXN_dayclose, all_p, on = "Date", how = "right")
all_p = pd.merge(data_MSFT_dayclose, all_p, on = "Date", how = "right")
all_p = pd.merge(data_QCOM_dayclose, all_p, on = "Date", how = "right")
all_p = pd.merge(data_exchange_chn, all_p, on = "Date", how = "right")
all_p = pd.merge(data_exchange_can, all_p, on = "Date", how = "right")
all_p = pd.merge(data_exchange_hk, all_p, on = "Date", how = "right")
all_p = pd.merge(data_exchange_euro, all_p, on = "Date", how = "right")

In [391]:
all_p.head(100)

,Euro exchange rate,Date,Hong Kong exchange rate,Canada exchange rate,China exchange rate,QCOM close price,MSFT close price,TXN close price,Unnamed: 0,APPL close price,...,aggression,horror,suffering,ridicule,irritability,deception,disappointment,negative_emotion,nervousness,appear
0,0.734322,2014-01-07,7.7542,1.0742,6.0507,73.239998,36.410000,42.700001,0,77.148575,...,1.552795e-03,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.552795e-03,1.552795e-03,0.000000e+00,0.000000e+00,1.0
1,0.730887,2014-01-14,7.7536,1.0934,6.0402,73.389999,35.779999,43.340000,1,78.055717,...,1.552795e-03,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.552795e-03,1.552795e-03,0.000000e+00,0.000000e+00,1.0
2,0.735078,2014-01-15,7.7543,1.0932,6.0459,74.510002,36.759998,43.540001,2,79.622856,...,1.552795e-03,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.552795e-03,1.552795e-03,0.000000e+00,0.000000e+00,1.0
3,NaN,2014-01-20,NaN,NaN,NaN,NaN,NaN,NaN,4,NaN,...,3.386960e-03,8.467401e-04,5.080440e-03,0.000000e+00,0.000000e+00,8.467401e-04,0.000000e+00,5.080440e-03,4.233700e-03,1.0
4,0.734538,2014-02-07,7.7586,1.1014,6.0629,74.410004,36.560001,41.950001,5,74.239998,...,2.680406e-03,4.299354e-04,2.727301e-03,1.476887e-04,0.000000e+00,1.095222e-03,9.606644e-04,3.613433e-03,1.558922e-03,3.0
5,0.732923,2014-02-10,7.7569,1.1048,6.0591,74.690002,36.799999,42.360001,6,75.570000,...,8.934686e-04,1.433118e-04,9.091003e-04,4.922956e-05,0.000000e+00,3.650741e-04,3.202215e-04,1.204478e-03,5.196407e-04,3.0
6,NaN,2014-02-17,NaN,NaN,NaN,NaN,NaN,NaN,7,NaN,...,2.325581e-03,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,2.325581e-03,0.000000e+00,1.0
7,0.730460,2014-02-20,7.7555,1.1104,6.0832,75.949997,37.750000,44.200001,8,75.878571,...,1.609525e-03,7.165589e-05,4.545501e-04,2.461478e-05,0.000000e+00,1.825371e-04,1.601107e-04,1.765030e-03,2.598203e-04,2.0
8,0.727590,2014-02-24,7.7591,1.1058,6.0982,75.430000,37.689999,44.450001,9,75.364288,...,8.047625e-04,3.582795e-05,2.272751e-04,1.230739e-05,0.000000e+00,9.126853e-05,8.005536e-05,8.825148e-04,1.299102e-04,2.0
9,0.726586,2014-03-03,7.7600,1.1090,6.1460,73.629997,37.779999,44.580002,11,75.394287,...,1.092512e-03,3.629795e-04,1.148834e-03,6.153695e-06,0.000000e+00,4.563427e-05,7.301588e-04,3.546847e-03,6.495509e-05,2.0


In [392]:

all_p.to_csv("close_all_news_extra.csv")

In [394]:
all_p = pd.read_csv("allnewsfinal.csv")

In [395]:
all_p = pd.merge(data_APPL_diff, all_p, on = "Date", how = "right")
all_p = pd.merge(data_TXN_diff, all_p, on = "Date", how = "right")
all_p = pd.merge(data_MSFT_diff, all_p, on = "Date", how = "right")
all_p = pd.merge(data_QCOM_diff, all_p, on = "Date", how = "right")
all_p = pd.merge(data_exchange_chn, all_p, on = "Date", how = "right")
all_p = pd.merge(data_exchange_can, all_p, on = "Date", how = "right")
all_p = pd.merge(data_exchange_hk, all_p, on = "Date", how = "right")
all_p = pd.merge(data_exchange_euro, all_p, on = "Date", how = "right")

In [399]:
all_p = all_p.drop("Unnamed: 0", axis = 1)
all_p = all_p.drop("APPL close price", axis = 1)

In [400]:
all_p.to_csv("different_all_news_extra.csv")